In [ ]:
!pip install reportlab

from flask import Flask, request, jsonify, render_template_string, send_file, session
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Only download once
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

app = Flask(__name__)

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# Define keywords
keywords = {
    "hello": ["hello", "hi", "hey", "hola", "नमस्ते", "नमस्कार"],
    "how are you": ["how", "are", "you", "कैसे", "हाल"],
    "what is your name": ["what", "is", "your", "name", "नाम", "क्या"],
    "what is ai": ["what", "is", "ai", "artificial", "intelligence", "कृत्रिम", "बुद्धिमत्ता", "ai"],
    "modi": ["who", "is", "modi", "narendra", "indian", "politician", "नरेंद्र", "मोदी", "भारतीय", "राजनीतिज्ञ", "मोदी", "मोदी"],
    "amit shah": ["who", "is", "amit", "shah", "indian", "politician", "अमित", "शाह", "भारतीय", "राजनेता"],
    # Add more keywords as needed
}

# Define responses
responses = {
    "hello": {
        "english": "Hello! How can I help you?",
        "hindi": "नमस्ते! मैं आपकी कैसे मदद कर सकता हूँ?"
    },
    "how are you": {
        "english": "I am just a bot, but I'm here to help you!",
        "hindi": "मैं एक बॉट हूँ, लेकिन मैं आपकी मदद के लिए यहाँ हूँ!"
    },
    "what is your name": {
        "english": "I am a simple chatbot.",
        "hindi": "मैं एक साधारण चैटबॉट हूँ।"
    },
    "what is ai": {
        "english": "Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, and self-correction.",
        "hindi": "कृत्रिम बुद्धिमत्ता (AI) मशीनों द्वारा मानव बुद्धि प्रक्रियाओं का अनुकरण है, विशेष रूप से कंप्यूटर सिस्टम द्वारा। इन प्रक्रियाओं में सीखना, तर्क करना और आत्म-सुधार शामिल है।"
    },
    "modi": {
        "english": "Narendra Modi is an Indian politician serving as the 14th and current Prime Minister of India since 2014. He was the Chief Minister of Gujarat from 2001 to 2014 and is a member of the Bharatiya Janata Party (BJP).",
        "hindi": "नरेंद्र मोदी भारत के 14वें और वर्तमान प्रधानमंत्री के रूप में सेवा कर रहे हैं। 2014 से वह गुजरात के मुख्यमंत्री थे और भारतीय जनता पार्टी (भाजपा) के सदस्य हैं।"
    },
    "amit shah": {
        "english": "Amit Shah is an Indian politician serving as the Minister of Home Affairs since 2019. He is a prominent leader of the Bharatiya Janata Party (BJP) and has held several key positions within the party.",
        "hindi": "अमित शाह भारत के गृह मंत्री के रूप में 2019 से सेवा कर रहे हैं। वह भारतीय जनता पार्टी (भाजपा) के प्रमुख नेता हैं और पार्टी के अन्यतम पदों पर कई महत्वपूर्ण पद धारित कर चुके हैं।"
    },
    # Add other responses here
    "default": {
        "english": "I'm sorry, I don't understand that.",
        "hindi": "मुझे खेद है, मैं इसे नहीं समझता।"
    }
}

app.secret_key = 'supersecretkey'

def get_conversation():
    if 'conversation' not in session:
        session['conversation'] = []
    return session['conversation']

def add_to_conversation(role, message):
    conversation = get_conversation()
    conversation.append({"role": role, "message": message})
    session['conversation'] = conversation

def get_response(user_input):
    user_input_lower = user_input.lower()

    if "in hindi" in user_input_lower:
        language = 'hindi'
        user_input_lower = user_input_lower.replace("in hindi", "").strip()
    elif "in english" in user_input_lower:
        language = 'english'
        user_input_lower = user_input_lower.replace("in english", "").strip()
    else:
        if any(char in 'अआइईउऊऋएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशषसह' for char in user_input_lower):
            language = 'hindi'
        else:
            language = 'english'

    tokens = preprocess(user_input_lower)

    matched_keywords = []
    for key, values in keywords.items():
        if any(token in values for token in tokens):
            matched_keywords.append(key)

    if matched_keywords:
        key = matched_keywords[0]  # Assuming the first matched keyword is the most relevant
        return responses[key][language]
    else:
        return responses["default"][language]

def wrap_text(text, max_width, p, font_name="Helvetica", font_size=12):
    """Wrap text to fit within a specified width."""
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        test_line = f"{current_line} {word}".strip()
        if p.stringWidth(test_line, font_name, font_size) <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    lines.append(current_line)
    return lines

def generate_pdf():
    # Register Devanagari font
    pdfmetrics.registerFont(TTFont('Devanagari', 'C:/Users/LENOVO/Downloads/TiroDevanagariHindi-Regular.ttf'))

    buffer = BytesIO()
    p = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter
    margin = 40
    max_width = width - 2 * margin
    y = height - margin

    for msg in get_conversation():
        if msg['role'] == 'user':
            p.setFillColorRGB(0, 0, 0)  # Black color for user message
            p.setFont("Helvetica", 12)
            lines = wrap_text(f"User: {msg['message']}", max_width, p)
        else:
            p.setFillColorRGB(0, 0, 1)  # Blue color for bot message
            bot_message = msg['message']
            if any(char in 'अआइईउऊऋएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशषसह' for char in bot_message):
                p.setFont("Devanagari", 12)
            else:
                p.setFont("Helvetica", 12)
            lines = wrap_text(f"Bot: {bot_message}", max_width, p)

        for line in lines:
            p.drawString(margin, y, line)
            y -= 20
            if y < margin:
                p.showPage()
                y = height - margin

    p.save()
    buffer.seek(0)
    return buffer

@app.route('/')
def home():
    return render_template_string('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot Interface</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        .container {
            max-width: 800px;
            margin: auto;
            padding: 20px;
        }
        .chat-container {
            border: 1px solid #ccc;
            border-radius: 5px;
            padding: 10px;
            margin-bottom: 20px;
            height: 400px;
            overflow-y: scroll;
        }
        .user-message {
            background-color: #f0f0f0;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 5px;
        }
        .bot-message {
            background-color: #007bff;
            color: white;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 5
px;
            text-align: right;
        }
        input[type="text"] {
            width: calc(100% - 70px);
            padding: 10px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        button {
            width: 60px;
            padding: 10px;
            border-radius: 5px;
            border: none;
            background-color: #007bff;
            color: white;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="chat-container" id="chat-container">
            <!-- Chat messages will be appended here -->
        </div>
        <input type="text" id="user-input" placeholder="Type your message...">
        <button onclick="sendMessage()">Send</button>
        <form action="/download_pdf" method="post">
            <button type="submit">Download PDF</button>
        </form>
    </div>

    <script>
        function appendMessage(role, message) {
            const chatContainer = document.getElementById('chat-container');
            const messageDiv = document.createElement('div');
            messageDiv.classList.add(role === 'user' ? 'user-message' : 'bot-message');
            messageDiv.textContent = message;
            chatContainer.appendChild(messageDiv);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function sendMessage() {
            const userInput = document.getElementById('user-input');
            const message = userInput.value;
            if (message.trim() === '') return;
            appendMessage('user', message);
            userInput.value = '';

            fetch('/chat', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ message: message })
            })
            .then(response => response.json())
            .then(data => {
                appendMessage('bot', data.response);
            });
        }
    </script>
</body>
</html>
    ''')

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    response = get_response(user_input)
    add_to_conversation('user', user_input)
    add_to_conversation('bot', response)
    return jsonify({'response': response})

@app.route('/download_pdf', methods=['POST'])
def download_pdf():
    pdf_buffer = generate_pdf()
    return send_file(pdf_buffer, as_attachment=True, download_name='conversation.pdf', mimetype='application/pdf')

if __name__ == '__main__':
    app.run(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
